In [10]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout,GridspecLayout,interact, interact_manual,IntProgress,VBox,HBox,Button, ButtonStyle, Output,FileUpload
from IPython.display import HTML, display, clear_output, FileLink, FileLinks
import os
import subprocess
import lxml
import base64
import hashlib
from typing import Callable
# need pandas version >= 1.3

# [DataPipe_signalement_docelec] Procédures automatisées de signalement de la documentation électronique

***

## Consignes d'utilisation

### Workflow FTF
- Exporter les données depuis FTF en format cvs
- **Veillez à bien mettre à jour xslt/tables_conversion.xsl si besoin**
- Déposer le fichier dans /source_files/ et faire tourner le workflow ftf
- Récupérer le fichier dans /result_files, puis
 - Vérifier encodage UTF-8 (sans BOM) 
 - double-zipper en tar.gz
- Pipe Primo delete_reload

### Workflow Numilog
- Exporter le fichier de notices Unimarc depuis le BO Numilog [https://admin.biblioaccess.com](https://admin.biblioaccess.com) (identifiants : cf wiki) → Mon catalogue → Télécharger vos notices en Unimarc
- Opérations de conversion dans MarcEdit
 - Encoder en UTF-8 : module MarcEditor -> menu File : ouvrir le fichier .mrc -> menu Edit : sélectionner "UTF-8 Character Map" (puis fermer la deuxième fenêtre) -> menu File : Save
 - Conversion Unimarc ISO 2709 séquentiel .mrc en Unimarc xml : module Marc Tools -> Choisir l'opération de transformation MARC21=>MARC21XML
- Déposer le fichier dans /source_files et faire tourner le workflow numilog
- Récupérer le fichier dans /result_files, puis
 - Vérifier encodage UTF-8 (sans BOM) 
 - double-zipper en tar.gz
- Pipe Primo delete_reload

### Worflow Cairn titre à titre
- Conversion Marc21 ISO 2709 séquentiel .mrc en Unimarc xml avec le module MarcTools de MarcEditor
 - Ouvrir MarcEditor -> Sélectionner Marc Tools -> Choisir l'opération de transformation MARC21=>MARC21XML
- Déposer le(s) fichier(s) dans /source_files et faire tourner le workflow cairn_titre_a_titre
- Récupérer le(s) fichier(s) dans /result_files, puis
 - Vérifier encodage UTF-8 (sans BOM) 
 - Zipper en tar
- Pipe Primo regular
- **Important : ne supprimer aucun fichier du serveur Aleph dans exlibris/aleph/aleph_export_2_primo/Cairn/**

### Workflow Cyberlibris
- Exporter le fichier de notices Unimarc depuis le BO Cyberlibris [https://extranet2.cyberlibris.com/index/login](https://extranet2.cyberlibris.com/index/login) (identifiants : cf wiki) → Onglet Notices
 - Choisir dans la liste Notices -> Complètes -> ISO :
   - Scholarvox_universite_emploi__metiers_et_formation_DATE_LA_PLUS_RECENTE.Pan
   - Scholarvox_universite_sciences_eco_gestion_DATE_LA_PLUS_RECENTE.Pan
- Conversion Unimarc ISO 2709 séquentiel .mrc en Unimarc xml avec le module MarcTools de MarcEditor
 - Ouvrir MarcEditor -> Sélectionner Marc Tools -> Choisir l'opération de transformation MARC21=>MARC21XML
- Déposer le(s) fichier(s) dans /source_files et faire tourner le pipe cyberlibris
- Récupérer le(s) fichier(s) dans /result_files, puis
 - Vérifier encodage UTF-8 (sans BOM) 
 - double-zipper en tar.gz
- Pipe Primo delete_reload
***

***

## Lancer le workflow

In [5]:
w_options = ['ftf', 'cairn_qsj', 'cairn_titre_a_titre', 'cyberlibris', 'numilog']
uploader = widgets.FileUpload()
spin = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    description='Loading:',
    bar_style='success', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
workflow = widgets.Dropdown(
    value='ftf', 
    options=w_options, 
    description='Choisir le workflow',
    style={'description_width': 'initial'},
    layout = Layout(width='350px')
)
filename = widgets.Text( 
    description='Nom du fichier source (déposée dans /source_files/)',
    value='par exemple : atoz_ftf_records.csv',
    style={'description_width': 'initial'},
    layout = Layout(width='600px')
)
tempfiles = widgets.Checkbox(
            value=False,
            description='Supprimer les fichiers temporaires de traitement',
            disabled=False,
        )
button = widgets.Button(description="Start",button_style='primary')
output = widgets.Output()

h1 =  widgets.HBox(
    [
        filename, 
        tempfiles
    ]
)

h2 = widgets.VBox(
    [
        button,
        spin, 
        output
    ]
)

display(workflow,h1,h2)

def file_path(relative_path):
    folder = os.path.dirname(os.path.abspath("__file__"))
    path_parts = relative_path.split("/")
    new_path = os.path.join(folder, *path_parts)
    return new_path

def on_button_clicked(b):
    with output:
        clear_output()
        if workflow.value == "ftf":
            spin.value = 5
            %run execute_workflow.py -w:ftf -f:{filename.value} 
            spin.value = 10
            if str(tempfiles.value) == "True":
                os.system("rm -rf temporary_files/")
                print('temp ftf file deleted')
            '''df = pd.read_csv("source_files/"+filename.value, sep=',', encoding='utf8')
            spin.value = 1
            #clean
            df = df.drop(atoz_cols_to_remove, axis=1).fillna("").replace("&", "&amp;")
            spin.value = 2
            # custom bsc cleaning
            df.drop(df[(df.PackageName == 'Business Source Complete') & ((df.ResourceType == 'Report') | (df.ResourceType == 'Book Series'))].index, inplace=True)
            spin.value = 3
            df.to_xml(path_or_buffer="temporary_files/atoz_export.xml", root_name='Resources', row_name='Resource', encoding='utf-8', xml_declaration=True, pretty_print=True, parser='lxml')
            spin.value = 4
            print(subprocess.run(["run_saxon.bat",file_path("xslt/atoztemp4primo.xsl"),file_path("temporary_files/atoz_export.xml"),file_path("result_files/atoz.xml")], shell=True, check=True, capture_output=True))
            spin.value = 5
            '''
            
        if workflow.value == "numilog":
            spin.value = 5
            %run execute_workflow.py -w:numilog -f:{filename.value}
            spin.value = 10
            
        if workflow.value == "cairn_titre_a_titre":
            spin.value = 5
            %run execute_workflow.py -w:cairn_titre_a_titre -f:{filename.value}
            spin.value = 10
            
        if workflow.value == "cyberlibris":
            spin.value = 5
            %run execute_workflow.py -w:cyberlibris -f:{filename.value}
            spin.value = 10

button.on_click(on_button_clicked) 

Dropdown(description='Choisir le workflow', layout=Layout(width='350px'), options=('ftf', 'cairn_qsj', 'cairn_…

***

***

## Récupérer le fichier résultat

Tous les fichiers résultats sont récupérables dans /result_files/ : cliquer sur "Voir les fichiers du répertoire /result_files" pour aficher la liste des fichiers (ou actualiser le liste)
- Voir le contenu : clic sur le lien du fichier
- Télécharger le fichier : clic droit sur le lien 
 - Firefox : -> Enregistrer la cible du lien sous ...
 - Chrome : -> Enregistrer le lien sous ...

In [17]:
filelinks_button = widgets.Button(description="Voir les fichiers du répertoire /result_files",
                                  button_style='success',
                                  layout = Layout(width='300px'))
filelinks_output = widgets.Output()
def on_flbutton_clicked(b):
    with filelinks_output:
        display(FileLinks("result_files/"))
filelinks_button.on_click(on_flbutton_clicked) 

display(filelinks_button,filelinks_output)

Button(button_style='success', description='Voir les fichiers du répertoire /result_files', layout=Layout(widt…

Output()

***